In [1]:
from fast_ml.utilities import reduce_memory_usage
from tqdm import tqdm

import matplotlib.pyplot as plt
import platform
if platform.system() == 'Darwin': #맥
        plt.rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': #윈도우
        plt.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Linux': # 구글 코랩
        plt.rc('font', family='Malgun Gothic') 
plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

df = pd.read_csv("./dataset/음식제거_최종.csv")

### 1. 특정 키워드 제거 
Target : 공모전, FIT

제거 전

In [2]:
df.head()

,id,관광지,url,촬영장소,검색 키워드
0,2778571,가리산 레포츠파크,http://tong.visitkorea.or.kr/cms2/website/71/2...,강원도 홍천군 두촌면,"가리산 레포츠파크,강원도 홍천군,가리산자연휴양림,테마파크,포레스트 어드벤처,가을,추..."
1,1859697,가마,http://tong.visitkorea.or.kr/cms2/website/97/1...,경상북도 영양군,가마
2,1038887,가면축제_롯데월드,http://tong.visitkorea.or.kr/cms2/website/87/1...,서울 롯데월드,"롯데월드 가면축제,마스크 퍼레이드,놀이동산,놀이공원,축제,FIT"
3,1055486,가산산성,http://tong.visitkorea.or.kr/cms2/website/86/1...,경상북도 칠곡군,가산산성
4,1054542,가실성당,http://tong.visitkorea.or.kr/cms2/website/42/1...,경상북도 칠곡군,칠곡 가실성당


제거 후

In [8]:
import re
total = []
before_keyword_cnt = []
after_keyword_cnt = []
blank_end = re.compile('$')

for i in range(df.shape[0]):
    temp = []
    keywords = df.loc[i,'검색 키워드'].split(',')
    before_keyword_cnt.append(len(keywords))
    for keyword in keywords:
        if(keyword.find('공모전') == -1 and keyword.find('입선') == -1 and keyword.find('FIT') == -1 and keyword != ''):
            temp.append(keyword.strip().replace('.','')) # 불필요한 공백, '.' 제거
    total.append(df.loc[i,['id','관광지','url','촬영장소']].to_list()+[','.join(temp)])
    after_keyword_cnt.append(len(temp))

print(f"제거 전 관광지 별 평균 키워드 수 : {round(np.mean(before_keyword_cnt),1)}")
print(f"제거 후 관광지 별 평균 키워드 수 : {round(np.mean(after_keyword_cnt),1)}")

제거 전 관광지 별 평균 키워드 수 : 5.5
제거 후 관광지 별 평균 키워드 수 : 5.2


In [9]:
filtered_df = pd.DataFrame(total,columns=list(df.columns))
filtered_df.head()

,id,관광지,url,촬영장소,검색 키워드
0,2778571,가리산 레포츠파크,http://tong.visitkorea.or.kr/cms2/website/71/2...,강원도 홍천군 두촌면,"가리산 레포츠파크,강원도 홍천군,가리산자연휴양림,테마파크,포레스트 어드벤처,가을,추..."
1,1859697,가마,http://tong.visitkorea.or.kr/cms2/website/97/1...,경상북도 영양군,가마
2,1038887,가면축제_롯데월드,http://tong.visitkorea.or.kr/cms2/website/87/1...,서울 롯데월드,"롯데월드 가면축제,마스크 퍼레이드,놀이동산,놀이공원,축제"
3,1055486,가산산성,http://tong.visitkorea.or.kr/cms2/website/86/1...,경상북도 칠곡군,가산산성
4,1054542,가실성당,http://tong.visitkorea.or.kr/cms2/website/42/1...,경상북도 칠곡군,칠곡 가실성당


### 2. 제거 이후 검색 키워드 빈도수

In [10]:
from collections import Counter

cr = Counter(','.join(filtered_df['검색 키워드'].values).split(','))
sorted_cr = sorted(cr.items(), key=lambda kv: kv[1], reverse=True)

In [11]:
sorted_cr[:10]

[('항공촬영', 307),
 ('드론사진', 305),
 ('드론촬영', 304),
 ('제주도', 240),
 ('절', 200),
 ('사찰', 197),
 ('바다', 174),
 ('전시관', 169),
 ('추경', 156),
 ('가을', 152)]

In [12]:
freq = np.array([i[1] for i in sorted_cr])
upper_1 = freq[freq>1]
freq_1 = freq[freq==1]

print(f"전체 키워드 수 : {len(freq)}")
print(f"2회 이상 발생 키워드 수 : {len(upper_1)} ({round(len(upper_1)/len(freq),2)*100}%)")
print(f"1회 발생 키워드 수 : {len(freq_1)} ({round(len(freq_1)/len(freq),2)*100}%)")

전체 키워드 수 : 9299
2회 이상 발생 키워드 수 : 1946 (21.0%)
1회 발생 키워드 수 : 7353 (79.0%)


### 2.1. 검색 키워드 단어 빈도수

In [13]:
word_cr = Counter(','.join(filtered_df['검색 키워드'].values).replace(',',' ').split(' '))
sorted_word_cr = sorted(word_cr.items(), key=lambda kv: kv[1], reverse=True)

In [14]:
sorted_word_cr[:10]

[('강원도', 459),
 ('전라남도', 324),
 ('항공촬영', 307),
 ('드론사진', 305),
 ('드론촬영', 304),
 ('경상북도', 299),
 ('제주도', 293),
 ('DMZ', 274),
 ('전라북도', 255),
 ('사찰', 216)]

In [15]:
freq = np.array([i[1] for i in sorted_word_cr])
upper_1 = freq[freq>1]
freq_1 = freq[freq==1]

print(f"전체 키워드 수 : {len(freq)}")
print(f"2회 이상 발생 키워드 수 : {len(upper_1)} ({round(len(upper_1)/len(freq),2)*100}%)")
print(f"1회 발생 키워드 수 : {len(freq_1)} ({round(len(freq_1)/len(freq),2)*100}%)")

전체 키워드 수 : 8836
2회 이상 발생 키워드 수 : 2804 (32.0%)
1회 발생 키워드 수 : 6032 (68.0%)


### 3. 지역 키워드 비중 확인
-> 높은 비중을 차지하는 키워드가 다수 존재하는 경우 편향된 추천 결과가 나타날 수 있을 것으로 생각되기 때문에, 높은 빈도를 나타내는 지역에 대한 정보가 전체 데이터에서 얼만큼의 비중을 차지하는지 우선적으로 파악

In [16]:
import re
region_pattern = re.compile('서울|부산|대구|인천|광주|대전|울산|세종|경기도|강원도|충청북도|충청남도|전라북도|전라남도|경상북도|경상남도|제주|담양|인사동|마포|홍대|공주|안동|울진인제|진주|청주|포천|화진포')

keyword_cnt = 0
region_keyword_list = []

for i in range(filtered_df.shape[0]):
    keywords = filtered_df.loc[i,'검색 키워드'].split(',')
    keyword_cnt += len(keywords)
    for keyword in keywords:
        if(not re.match(region_pattern,keyword) is None and keyword != ''):
            region_keyword_list.append(keyword)

print(f"지역 키워드 수 : {len(region_keyword_list):,}개 (전체 {keyword_cnt:,} 개 중 {round(len(region_keyword_list)/keyword_cnt,2)*100:,}%)")
print(f"추출 지역 키워드 샘플 : {region_keyword_list[:5]}")

지역 키워드 수 : 3,945개 (전체 22,144 개 중 18.0%)
추출 지역 키워드 샘플 : ['강원도 홍천군', '전라북도전주시', '전라북도 전주시', '경상남도 합천군', '경상북도 성주군']


### 3.1. 지역 키워드 제거

In [17]:
keywords

['from the earth',
 '제주특별자치도 제주시',
 '제주도',
 '제주오름',
 '아부오름',
 '분화구',
 '자연',
 '드론촬영',
 '드론사진',
 '항공촬영']

In [20]:
import re
region_pattern = re.compile('서울|부산|대구|인천|광주|대전|울산|세종|경기도|강원도|충청북도|충청남도|전라북도|전라남도|경상북도|경상남도|제주|담양|인사동|마포|홍대|공주|안동|울진인제|진주|청주|포천|화진포')

before_keyword_cnt = []
after_keyword_cnt = []
total = []

for i in range(filtered_df.shape[0]):
    temp = []

    keywords = filtered_df.loc[i,'검색 키워드'].split(',')
    before_keyword_cnt.append(len(keywords))
    
    for keyword in keywords:
        if(re.match(region_pattern,keyword) is None and keyword != ''):
            temp.append(keyword)
    total.append(df.loc[i,['id','관광지','url','촬영장소']].to_list()+[','.join(temp)])
    after_keyword_cnt.append(len(temp))
    
print(f"제거 전 관광지 별 평균 키워드 수 : {round(np.mean(before_keyword_cnt),1)}")
print(f"제거 후 관광지 별 평균 키워드 수 : {round(np.mean(after_keyword_cnt),1)}")

제거 전 관광지 별 평균 키워드 수 : 5.2
제거 후 관광지 별 평균 키워드 수 : 4.3


지역 키워드 제거 후 평균 키워드 수가 0.9 개 변화한 것으로 보아, 기본적으로 각 관광지는 지역에 해당하는 검색 키워드를 가지고 있는 것으로 보인다

In [21]:
del_reg_df = pd.DataFrame(total,columns=list(df.columns))
del_reg_df

,id,관광지,url,촬영장소,검색 키워드
0,2778571,가리산 레포츠파크,http://tong.visitkorea.or.kr/cms2/website/71/2...,강원도 홍천군 두촌면,"가리산 레포츠파크,가리산자연휴양림,테마파크,포레스트 어드벤처,가을,추경,단풍,드론촬..."
1,1859697,가마,http://tong.visitkorea.or.kr/cms2/website/97/1...,경상북도 영양군,가마
2,1038887,가면축제_롯데월드,http://tong.visitkorea.or.kr/cms2/website/87/1...,서울 롯데월드,"롯데월드 가면축제,마스크 퍼레이드,놀이동산,놀이공원,축제"
3,1055486,가산산성,http://tong.visitkorea.or.kr/cms2/website/86/1...,경상북도 칠곡군,가산산성
4,1054542,가실성당,http://tong.visitkorea.or.kr/cms2/website/42/1...,경상북도 칠곡군,칠곡 가실성당
...,...,...,...,...,...
4221,2620374,Through My Sunglasses,http://tong.visitkorea.or.kr/cms2/website/74/2...,서울특별시 종로구,"Through My Sunglasses,조계사,절,사찰,선글라스,안경,연등"
4222,1136789,Toy(북카페),http://tong.visitkorea.or.kr/cms2/website/89/1...,서울,"토이,북카페,책"
4223,2504217,Umbrella mania,http://tong.visitkorea.or.kr/cms2/website/17/2...,서울특별시 마포구 서교동,"특별상,Umbrella mania,메세나폴리스,우산"
4224,1136898,VOOK'S (북카페),http://tong.visitkorea.or.kr/cms2/website/98/1...,서울,책카페


In [22]:
for i in range(del_reg_df.shape[0]):
    keywords = del_reg_df.loc[i,'검색 키워드']
    for keyword in keywords:
        if(keyword == ''):
            print(keywords)

## 3.2. 지역 키워드 제거 후 빈도수 확인

In [90]:
word_cr = Counter(','.join(del_reg_df['검색 키워드'].values).replace(',',' ').split(' '))
sorted_word_cr = sorted(word_cr.items(), key=lambda kv: kv[1], reverse=True)

In [91]:
sorted_word_cr[:10]

[('항공촬영', 307),
 ('드론사진', 305),
 ('드론촬영', 304),
 ('DMZ', 274),
 ('사찰', 213),
 ('가을', 207),
 ('절', 202),
 ('전시관', 180),
 ('바다', 177),
 ('야경', 175)]

In [25]:
freq = np.array([i[1] for i in sorted_word_cr])
upper_1 = freq[freq>1]
freq_1 = freq[freq==1]

print(f"전체 키워드 수 : {len(freq)}")
print(f"2회 이상 발생 키워드 수 : {len(upper_1)} ({round(len(upper_1)/len(freq),2)*100}%)")
print(f"1회 발생 키워드 수 : {len(freq_1)} ({round(len(freq_1)/len(freq),2)*100}%)")

전체 키워드 수 : 8174
2회 이상 발생 키워드 수 : 2445 (30.0%)
1회 발생 키워드 수 : 5729 (70.0%)


In [27]:
del_reg_df.to_csv('./dataset/음식,지역 제거.csv',index=False,encoding='utf-8-sig')

### 빈도수 상위 단어 10에 대한 비중 확인

In [30]:
keyword_list = np.array(','.join(del_reg_df['검색 키워드'].values).replace(',',' ').split(' '))

In [83]:
keyword_df.loc[~keyword_df['검색 키워드'].isin([i[0] for i in sorted_word_cr[:10]])].shape[0], keyword_df.loc[keyword_df['검색 키워드'].isin([i[0] for i in sorted_word_cr[:10]])].shape[0]

(21533, 2344)

In [112]:
teop_10 = [i[0] for i in sorted_word_cr[:10]]
keyword_df = pd.DataFrame(keyword_list,columns=['검색 키워드'])
except_keyword = []
top_10_keyword = []
for i in range(keyword_df.shape[0]):
    if (keyword_df.loc[i,'검색 키워드'] not in teop_10):
        except_keyword.append(keyword_df.loc[i,'검색 키워드'])
    else:
        top_10_keyword.append(keyword_df.loc[i,'검색 키워드'])

print(f"빈도수 상위 키워드 10개를 제외한 검색 키워드 종류 수 : {len(set(except_keyword))}")
print(f"빈도수 상위 키워드 10개 수 : {len(top_10_keyword):,}개 ({round(len(top_10_keyword)/len(except_keyword)*100,2)}%)")

빈도수 상위 키워드 10개를 제외한 검색 키워드 종류 수 : 8164
빈도수 상위 키워드 10개 수 : 2,344개 (10.89%)


In [105]:
except_word_cr = Counter(except_keyword)
except_sorted_word_cr = sorted(except_word_cr.items(), key=lambda kv: kv[1], reverse=True)

In [113]:
except_sorted_word_cr[:10]

[('추경', 159),
 ('관광', 148),
 ('유네스코', 146),
 ('단풍', 142),
 ('봄', 133),
 ('사적', 129),
 ('안보관광지', 128),
 ('투어', 127),
 ('산책로', 110),
 ('기획사진', 110)]